Same model, randomised. We know that the values for the variables follow a uniform distribution between 0 and 1, and the data that we see are the product of multiplication of each factor by this Protein,Carbohydrats and Fats value. Then we make a guess of what distribution do the parameters follow, an ubiased guess would be that they follow a normal(1,100). Then the ATP is the deterministic variable in the model, because it is determined by the other three.
The error can be a normal with mean 0 and std 10.

We assign observed data to P,C,F and ATP, se we have to assign determinitic values to these.

In [3]:
import pymc3 as pm
import pandas as pd
import numpy as np


You can find the C code in this temporary file: /tmp/theano_compilation_error_9ey1by8z


AttributeError: module 'theano' has no attribute 'gof'

In [ ]:
data = pd.read_csv("diet_and_atp.tsv", header=[0], sep="\t")
model_input = data[["Protein","Carbohydrates","Fat"]]
model_output = data[["ATP"]]

In [ ]:
with pm.Model() as model:
    std = pm.Uniform("std", 0, 100)
    P = pm.Uniform("P",0,1)
    C = pm.Uniform("C",0,1)
    F = pm.Uniform("F",0,1)
    p = pm.Normal("p", mu = 1, sd = 100)
    c = pm.Normal("c", mu = 1, sd = 100)
    f = pm.Normal("f", mu = 1, sd = 100)
    
    data_prot = pm.Deterministic("proteins",p*P)
    data_carb = pm.Deterministic("carbohydrates",c*C)
    data_fat = pm.Deterministic("fat",f*F)
    mean = pm.Deterministic("mean", p*P+c*C+f*F)
    atp = pm.Normal("ATP", mu = mean, sd = std, observed = data["ATP"])
    
    trace = pm.sample(10000, step=pm.Metropolis()) #aquí exploramos el modelo con el algoritmo de metropolis
    burned_trace = trace[20000:]  


In [ ]:
#En este bloque le damos al modelo todos los parámetros
with pm.Model() as model:
    std = pm.Uniform("std", 0, 100) #100 es solo un guess
    
    beta = pm.Normal("beta", mu=0, sd=100) #estos parametros tiene una distribución normal, le damos una sd muy amplia
    alpha = pm.Normal("alpha", mu=0, sd=100) #decimos que estos dos siguen una normal porque en la grafica anterior hemos visto que los valores están entorno a 0, pero esto tambien es un guess, un guess ligeramente informado por la exploración previa de los datos,
    #podríamos probar otros guesses iniciales pero llegaremos a conclusiones más precisas si nuestros guesses están "biased" o, dicho de otra forma, informados
    
    # Deterministic variables that depends of random variables are set like this
    mean = pm.Deterministic("mean", alpha + beta*X) #la linea que quiero predecir, X son los valores que establecimos en el bloque anterior
    
    # We want to adjust mean and std and we have the observed data, we link them like this.
    obs = pm.Normal("obs", mu=mean, sd=std, observed=Y) #Y es el return, decimos que sigue una distribución normal entorno a la media,
    
    trace = pm.sample(100000, step=pm.Metropolis()) #aquí exploramos el modelo con el algoritmo de metropolis
    burned_trace = trace[20000:] 

In [ ]:

pm.plots.traceplot(trace=burned_trace, varnames=["std", "beta", "alpha"])
pm.plot_posterior(trace=burned_trace, varnames=["std", "beta", "alpha"], kde_plot=True);